<a href="https://colab.research.google.com/github/ngastellu/ac-bo-hackathon/blob/main/additive_mutations_nico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install 'transformers[torch]'
!pip install datasets
!pip install mdanalysis
!pip install ax-platform
!pip install -q git+https://github.com/ORNL/affinity_pred.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [9]:
import numpy as np
np.object = object
np.bool = bool
np.int = int
import torch
#from torch.utils.data import Dataset
from datasets import load_dataset, Dataset
import datasets
from transformers import AutoTokenizer, Trainer
from transformers import EvalPrediction
from transformers import TrainingArguments

from huggingface_hub import hf_hub_download

map_location=torch.device('cpu')
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

import pandas as pd
import numpy as np
import re


# the transformer model
from affinity_pred.model import EnsembleSequenceRegressor

# the Bayesian optimization package
from ax import ChoiceParameter, RangeParameter
from ax.service.ax_client import AxClient, ObjectiveProperties

# Code that identifies binding residues on protein
from get_close_resids import binding_resids

import warnings
# Filter out the specific FutureWarning related to the `device` argument in transformers
warnings.filterwarnings("ignore", message="The `device` argument is deprecated and will be removed in v5 of Transformers.", category=FutureWarning, module="transformers.modeling_utils")



In [10]:
class predictor():
    seq_model_name = "Rostlab/prot_bert_bfd"
    seq_tokenizer = AutoTokenizer.from_pretrained(seq_model_name, do_lower_case=False)
    smiles_model_name = 'mossaic-candle/regex-gb-2021'
    smiles_tokenizer =  AutoTokenizer.from_pretrained(smiles_model_name)
    # the maximum length of a protein seqeuence (for inference)
    max_seq_length=512
    # the maximum length of a SMILES sequence
    max_smiles_length=256
    trainer = ""
    mod = ""
    def compute_metrics(p: EvalPrediction):
        preds_list, out_label_list = p.predictions, p.label_ids

        return {
            "mse": mean_squared_error(out_label_list, preds_list),
            "mae": mean_absolute_error(out_label_list, preds_list),
            "spearman_rho": spearmanr(out_label_list, preds_list).correlation,
        }

    def __init__(self):
        #self.testseq = testseq
        self.trainer = Trainer(
            model_init=self.model_init,                  # the instantiated Transformers model to be trained
            compute_metrics = self.compute_metrics,        # evaluation metric
            args=TrainingArguments(per_device_eval_batch_size=1, # reduce if running out of memory
                                   output_dir='results'),
                                   )
    def expand_seqs(self,seqs):
        """expand a FASTA sequence, fixing by removing U,Z,O,B with X"""
        input_fixed = ["".join(seq.split()) for seq in seqs]
        input_fixed = [re.sub(r"[UZOB]", "X", seq) for seq in input_fixed]
        return [list(seq) for seq in input_fixed]

    def encode(self,item,tolist=False):
        """encodes a {seq:"",smiles/smiles_can:""}
        for use in the model
        """
        seq_encodings = self.seq_tokenizer(self.expand_seqs(item['seq'])[0],
                                      truncation=True,
                                      padding='max_length',
                                      max_length=self.max_seq_length,
                                      is_split_into_words=True)

        # use RDkit canonical SMILES
        if 'smiles_can' in item:
            smiles = item['smiles_can'][0]
        else:
            smiles = item['smiles'][0]

        smiles_encodings = self.smiles_tokenizer(smiles,
                                            padding='max_length',
                                            truncation=True,
                                            max_length=self.max_smiles_length)

        # concatenate the two different inputs
        if tolist:
            item['input_ids'] = [torch.cat([torch.tensor(seq_encodings['input_ids']),
                                        torch.tensor(smiles_encodings['input_ids'])]).tolist()]
            item['attention_mask'] = [torch.cat([torch.tensor(seq_encodings['attention_mask']),
                                        torch.tensor(smiles_encodings['attention_mask'])]).tolist()]
        else:
            item['input_ids'] = [torch.cat([torch.tensor(seq_encodings['input_ids']),
                                            torch.tensor(smiles_encodings['input_ids'])])]
            item['attention_mask'] = [torch.cat([torch.tensor(seq_encodings['attention_mask']),
                                        torch.tensor(smiles_encodings['attention_mask'])])]
        return item

    def model_init(self):
        """
        loads the model out from huggingface
        """
        model = EnsembleSequenceRegressor(self.seq_model_name, self.smiles_model_name, max_seq_length=self.max_seq_length)

        # select one of the following ensemble members
        model_name = 'regex_1'
        #model_name = 'regex_2'
        #model_name = 'regex_3'
        #model_name = 'regex_4'
        #model_name = 'regex_5'

        # download the weights
        fn = hf_hub_download(repo_id="jglaser/affinity_pred_{}".format(model_name), filename='pytorch_model.bin')

        checkpoint = torch.load(fn,map_location=torch.device('cpu'))
        #get rid of the following keys: seq_model.embeddings.position_ids and smiles_model.embeddings.position_ids
        # as they are not supported in transformers>4.30.2 (the author trained on older version of transformers)
        rchck = {}
        for key in checkpoint.keys():
            if key in ['seq_model.embeddings.position_ids','smiles_model.embeddings.position_ids']:
                pass
            else:
                rchck.update({key:checkpoint[key]})
        model.load_state_dict(rchck)
        return model

    class AffinityDataset(Dataset):
        def __init__(self, dataset):
            self.dataset = dataset

        def __getitem__(self, idx):
            item = self.dataset[idx]

            #affinity = item['neg_log10_affinity_M']
            affinity = item['affinity']
            item['labels'] = float(affinity)

            # drop the non-encoded input
            item.pop('smiles')
            item.pop('smiles_can')
            item.pop('seq')
            item.pop('neg_log10_affinity_M')
            item.pop('affinity_uM')
            item.pop('affinity')
            return item

        def __len__(self):
            return len(self.dataset)




    def preloaded_to_oneoff(item):
        rdict = {'seq':item['seq'],"smiles":item['smiles'],'affinity_uM':0,'neg_log10_affinity_M':0,'affinity':0}
        return Dataset.from_pandas(pd.DataFrame.from_dict(encode(rdict,tolist=True)))
    def predict(self,testseq):
        item = self.encode(testseq,tolist=True)
        pdf = pd.DataFrame.from_dict(item)
        myds = Dataset.from_pandas(pdf)
        return self.trainer.predict(myds)

#FINISH SETUP THINGS

#loading the initial model will take some time, so keep it in memory until job is done
predictor = predictor()

#FEED THE y.predictions (from the docs are -log_10(muaffinity)) to the BO. INPUT INTO PREDICTOR.predict is a dict of {'seq':"FASTA(?) SEQ", "smiles":"SMILES OF A SMALL LIGAND"}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at mossaic-candle/regex-gb-2021 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define starting protein, ligand, and the residues that bind to the ligand
fasta_str_f = 'MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW'
# fasta = list(fasta_str_f)

pdb_file = '5tzo.pdb'
ligand_resname = '7V7'
rcut = 5.0 #angstroms

binding_residues = binding_resids(pdb_file,ligand_resname,rcut)
print(binding_residues)

axbo_params=[
    {
        "name": "position",
        "type": "choice",
        "values": list(binding_residues),
        "value_type": "int"
    },
    {
        "name": "mutation",
        "type": "choice",
        "values": ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"],
    }
]

def evaluate_protein_BA(fasta_seq, target_smiles, predictor_obj):
    testseq = {'seq': [fasta_seq], 'smiles': [target_smiles]}
    # Get predictions from the predictor
    prediction_result = predictor_obj.predict(testseq)
    #print("Prediction Result:", prediction_result)
    score_bo = prediction_result.predictions * -1

    return score_bo

def evaluate_mutation_BA(parameters, fasta,  target_smiles, predictor_obj):
    position = int(parameters["position"])
    mutation = parameters["mutation"]

    new_fasta = list(fasta[:])

    # Perform single-point mutation
    new_fasta[position] = mutation

    # Convert list to string to form the sequence
    mutated_sequence = ''.join(new_fasta)
    score_bo = evaluate_protein_BA(mutated_sequence, target_smiles, predictor_obj)
    print('Mutated:', mutation, position, "KD (uM):", 10 ** (score_bo), "Predictions:", -score_bo, "Mutated Sequence:", mutated_sequence)
    return score_bo


def additive_BO(axclient, fasta_init, target_smiles="CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3", predictor_obj=predictor, max_mutations=3, max_bo_steps=100, max_bo_loops=10,early_break=True):

    # get binding affinity of starting protein
    initial_score = evaluate_protein_BA(fasta_init, target_smiles, predictor_obj)
    print("Initial score = ", initial_score)

    print("Starting BO loop:")

    # initialise variables for BO search
    nmutations = 0
    ntries = 0

    ref_fasta = fasta_init[:]
    ref_score = initial_score
    accepted_mutations = {'position':[], "mutation": [], 'scores':[]}

    # prepare to save output from each BO iteration if early_break=False (see below for explanation of `early_break`)
    if not early_break:
      mutations = np.zeros((max_mutations, max_bo_steps), dtype='str')
      positions = np.zeros((max_mutations, max_bo_steps), dtype='int')
      scores = np.zeros((max_mutations, max_bo_steps), dtype='float')

    # BO search loop starts here
    while nmutations < max_mutations and ntries < max_bo_loops:
      print(f'~~~~~ Starting BO loop nb {ntries+1} ~~~~~')
      for i in range(max_bo_steps):
        new_params, itrial = axclient.get_next_trial() #get new mutation
        new_score = evaluate_mutation_BA(new_params,ref_fasta,target_smiles,predictor_obj) #evaluate new mutation
        axclient.complete_trial(itrial, raw_data=new_score) #update model

        position = int(new_params["position"])
        mutation = new_params["mutation"]

        if new_score > ref_score and early_break:
          # `early_break=True` interrupts the BO search as soon as a better candidate is found
          # if we have something better than the initial protein, interrupt BO, and save changes
          print(f'******** Mutation accepted! ********')
          new_fasta = list(ref_fasta[:])
          new_fasta[position] = mutation
          ref_fasta = ''.join(new_fasta)
          nmutations += 1
          accepted_mutations['position'].append(position)
          accepted_mutations['mutation'].append(mutation)
          break

        if not early_break:
          # if all BO trials are to be run, save output from each trial
          positions[nmutations,i] = position
          mutations[nmutations,i] = mutation
          scores[nmutations,i] = new_score


      if not early_break:
        # find the best scoring mutation: if better than current FASTA, save changes
        ibest = np.argmax(scores)
        best_score = scores[nmutations,ibest]
        if best_score > ref_score:
          print(f'******** Mutation accepted! ********')
          best_pos = positions[nmutations,ibest]
          best_mut = mutations[nmutations,ibest]
          print(f"Mutate site {best_pos}: {best_mut}")
          new_fasta = list(ref_fasta[:])
          new_fasta[best_pos] = best_mut
          ref_fasta = ''.join(new_fasta)
          nmutations += 1
          accepted_mutations['position'].append(position)
          accepted_mutations['mutation'].append(mutation)

      ntries += 1

    # do final scoring
    print("\nDone with BO loops! Evaluating final sequence...")
    final_score = evaluate_protein_BA(ref_fasta, target_smiles, predictor_obj)
    print("Final score = ", final_score)

    # output results; if early_break=False, also return each BO trial's binding score
    if early_break:
      return ref_fasta, final_score, accepted_mutations
    else:
      return ref_fasta, final_score, accepted_mutations, scores




# set up BO run
axclient = AxClient(verbose_logging=False)

axclient.create_experiment(
    name = "5TZO binding affinity",
    parameters = axbo_params,
    objectives = {"neg pKD": ObjectiveProperties(minimize=False)}
)

# run the BO
final_fasta, final_score, mutations = additive_BO(axclient,fasta_str_f, early_break=False)






/usr/local/lib/python3.10/dist-packages/ax/core/parameter.py:594: UserWarning: `is_ordered` is not specified for `ChoiceParameter` "position". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.
  warn(
/usr/local/lib/python3.10/dist-packages/ax/core/parameter.py:594: UserWarning: `sort_values` is not specified for `ChoiceParameter` "position". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.
  warn(
[INFO 03-29 01:16:40] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter mutation. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/usr/local/lib/python3.10/dist-packages/ax/core/parameter.py:594: UserWarning: `is_ordered` is not 

[  7   9  35  36  37  63  64  65  67  69  71  80  82  89  90  91  92 108
 110 111 112 116 117 118 120 125 127 129 131 166 172 173 174]


Initial score =  0.24744641780853271
Starting BO loop:
~~~~~ Starting BO loop nb 1 ~~~~~


Mutated: M 111 KD (uM): 0.9976891391300677 Predictions: 0.0010047554969787598 Mutated Sequence: MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTMYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW


Mutated: E 166 KD (uM): 1.4512639174999906 Predictions: -0.16174639761447906 Mutated Sequence: MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYEVMATAGYQSSGSSNVTVW


...

Mutated: A 166 KD (uM): 1.6824324440764162 Predictions: -0.2259376347064972 Mutated Sequence: MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYAVMATAGYQSSGSSNVTVW


Mutated: Q 7 KD (uM): 1.7882286475127178 Predictions: -0.2524230480194092 Mutated Sequence: MSTDYWLQFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW


Mutated: K 7 KD (uM): 1.5620468488504613 Predictions: -0.19369405508041382 Mutated Sequence: MSTDYWLKFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW


Mutated: Q 174 KD (uM): 1.7678540907627247 Predictions: -0.24744641780853271 Mutated Sequence: MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYVVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW


Mutated: D 80 KD (uM): 1.70993368505837 Predictions: -0.23297926783561707 Mutated Sequence: MSTDYWLNFTDGGGIVNAVNGSGGNYSVNWSNTGSFVVGKGWTTGSPFRTINYNAGVWAPNGWGALALVGWTRSPLIAYYDVDSWGTARWTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATAGYQSSGSSNVTVW
